In [2]:
import torch

In [2]:
x = torch.rand(2,2).requires_grad_(True) 
m = torch.rand(1,2)

In [3]:
dist = torch.cdist(x,m)
print(dist.shape)

torch.Size([2, 1])


In [4]:
for i in range(2):
    dist[i,0].backward(retain_graph=True)
    print(x.grad)

tensor([[-0.5339,  0.8456],
        [ 0.0000,  0.0000]])
tensor([[-0.5339,  0.8456],
        [ 0.7553,  0.6554]])


In [5]:
from src.densities import GMM
x = torch.rand(2,2).requires_grad_(True).float()
m = torch.rand(1,2).float()
w = torch.tensor([1]).float()
gmm = GMM(m,w)

In [6]:
y = gmm.eval_batch(x)
print(y.shape)

torch.Size([2])


In [7]:
for i in range(2):
    y[i].backward(retain_graph=True)
   


In [8]:
print((x-m)/x.grad)

tensor([[-6.2973, -6.2973],
        [-6.8855, -6.8855]], grad_fn=<DivBackward0>)


In [9]:
from src.kde import GaussianKDE as KDE

In [10]:
kde = KDE(1/5)
X = torch.rand(10,2).float()
R = torch.rand(2,2).float().requires_grad_(True)
f_hat = kde.estimate_batch(R,X)
print(f_hat)

tensor([0.0066, 1.6359], grad_fn=<MulBackward0>)


In [11]:
for i in range(2):
    f_hat[i].backward(retain_graph=True)
    print(R.grad)
    R.grad.zero_()

tensor([[-0.1172,  0.1835],
        [ 0.0000,  0.0000]])
tensor([[ 0.0000,  0.0000],
        [-1.1560, -4.5386]])


In [12]:
m_dot = torch.randn(1,2)
f_d = torch.randn(10,1)

# multiply each element of m_dot with each element of f_d
feed_forward = f_d@m_dot
feed_forward.shape

torch.Size([10, 2])

In [13]:
m0 = torch.randn((1,2))
m1 = torch.randn((1,2))
t= torch.tensor(0.5,requires_grad=True)
h = t/2
m = ((1-h)*m0 + t*m1)
m[0,0].backward(retain_graph=True)
print(t.grad)
print(m1-m0)

tensor(1.7206)
tensor([[2.0343, 1.3475]])


In [14]:
B = 10
k = 5
x = torch.randn(B,2)
w = torch.randn(k)
e = torch.randn(B,k)


In [15]:
y = e*w


In [16]:
for i in range(10):
    for j in range(5):
        d = e[i,j]*w[j]-y[i,j]
        print(d)

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)


In [17]:
mdot = torch.rand(k,2)
v = y@mdot
v.shape

torch.Size([10, 2])

In [6]:
N = 5

x = torch.randn(N,3, requires_grad=True)
y = x.pow(2).sum(1)
I_N = torch.eye(N)
# Sequential approach
jacobian= torch.autograd.grad(y, x,torch.ones(N))[0]
print(jacobian.shape) 
print(jacobian-2*x)               

torch.Size([5, 3])
tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], grad_fn=<SubBackward0>)


In [45]:
B = 15
x = torch.rand(B,2)
w = torch.rand(N)
e = torch.rand(B,N)
m0 = torch.rand(N,1)
m1  = torch.rand(N,1)
t = torch.tensor(0.5,requires_grad=True)
m = (1-t)*m0 + t*m1
ff = torch.zeros(B,1)
for i in range(B):
    ff[i]=torch.autograd.grad(m[:,0],t,e[i],retain_graph=True)[0]
print(ff.shape)


torch.Size([15, 1])


In [57]:
from src.densities import GMM
B = 300
N = 5353
x = torch.rand(B,2).requires_grad_(True)
m0 = torch.rand(N,2)
m1  = torch.rand(N,2)
t = torch.tensor(0.5,requires_grad=True)
m = (1-t)*m0 + t*m1
w = torch.rand(N)
w/=w.sum()
gmm = GMM(m.detach(),w)


In [58]:
f_d = gmm.eval_batch(x)
comps = gmm.get_components_batch(x)
print(f_d.shape)
print(comps.shape)

torch.Size([300])
torch.Size([300, 5000])


In [62]:
ff = torch.zeros_like(x)
for i in range(B):
    for j in range(2):
        ff[i,j]=torch.autograd.grad(m[:,j],t,comps[i],retain_graph=True)[0]
    


: 

In [61]:
m_dot = torch.zeros_like(m)
for i in range(m.shape[0]):
    for j in range(m.shape[1]):
        m[i,j].backward(retain_graph=True)
        m_dot[i,j] = t.grad
        t.grad.zero_()
feed_foward = comps@m_dot

tensor([[-5.5879e-09, -6.9849e-10],
        [ 1.8626e-09,  6.9849e-10],
        [-3.7253e-09, -1.6298e-09],
        [ 0.0000e+00, -3.4925e-09],
        [-3.7253e-09, -2.7940e-09],
        [ 5.5879e-09, -4.6566e-10],
        [ 1.8626e-09, -3.7253e-09],
        [ 5.5879e-09,  3.7253e-09],
        [ 6.5193e-09, -1.8626e-09],
        [-5.5879e-09, -3.2596e-09]], grad_fn=<SubBackward0>)


In [17]:
import torch
from src.gmminterpolator import GMMInterpolator
gmminter = GMMInterpolator.walking_man(0,20,(64,64))
gmminter.load_coeff('resolution64')

In [5]:
print(gmminter.ms[0].shape)
print(gmminter.PIs[0].shape)

torch.Size([4096, 2])
torch.Size([4096, 4096])


In [26]:
p0 = gmminter.PIs[0]
m0 = gmminter.ms[0]
m1 = gmminter.ms[1]
idx = (p0.flatten()>0)

In [27]:
a = torch.tensor([[1,2],[3,4],[5,6]])
print('repeat:')
print(a.repeat(2,1))
print('repeat interleave')
print(a.repeat_interleave(2,0))
c = torch.tensor([[1,2],[3,4]])
print(c)
print(c.flatten())


repeat:
tensor([[1, 2],
        [3, 4],
        [5, 6],
        [1, 2],
        [3, 4],
        [5, 6]])
repeat interleave
tensor([[1, 2],
        [1, 2],
        [3, 4],
        [3, 4],
        [5, 6],
        [5, 6]])
tensor([[1, 2],
        [3, 4]])
tensor([1, 2, 3, 4])


In [28]:
m0_ext = m0.repeat_interleave(p0.size(0),0)[idx]
print(m0_ext.shape)
m1_ext = m1.repeat(p0.size(1),1)[idx]
print(m1_ext.shape)

torch.Size([5353, 2])
torch.Size([5353, 2])


In [32]:
import torch
from src.densities import GMM
B = 300
x = torch.rand(B,2)
t = torch.tensor(0.5,requires_grad=True)
m = (1-t)*m0_ext + t*m1_ext
gmm = GMM(m.detach(),p0.flatten()[idx])
f_d = gmm.eval_batch(x)
comps = gmm.get_components_batch(x)


RuntimeError: expected scalar type Float but found Double